In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import torch
import math

from modules.util.util import importstr
from modules.util.logconf import logging
log = logging.getLogger('nb')

In [3]:
def run(app, *argv):    
    torch.cuda.empty_cache()
    
    argv = list(argv)
    log.info("Running: {}({!r}).main()".format(app, argv))
    
    app_cls = importstr(*app.rsplit('.', 1))
    app_cls(argv).main()
    
    log.info("Finished: {}.{!r}.main()".format(app, argv))

In [4]:
# width_irc=(16,128,128)
# run('prepcache.CovidPrepCacheApp', f'--num-workers={8}', f'--batch-size={20}', '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}')

2020-12-07 04:56:41,940 INFO     pid:22723 nb:005:run Running: prepcache.CovidPrepCacheApp(['--num-workers=8', '--batch-size=20', '--width-irc', '16', '128', '128']).main()
2020-12-07 04:56:44,332 INFO     pid:22723 prepcache:039:main Starting CovidPrepCacheApp, Namespace(batch_size=20, num_workers=8, width_irc=['16', '128', '128'])
100%|██████████| 10/10 [02:27<00:00, 14.72s/it]
2020-12-07 04:59:11,832 INFO     pid:22723 nb:010:run Finished: prepcache.CovidPrepCacheApp.['--num-workers=8', '--batch-size=20', '--width-irc', '16', '128', '128'].main()


## Basic Unet 3D

In [4]:
batch_size = 2
epochs = 50
steps_per_epoch=200
val_cadence = 5
recall_priority = 0
unet_depth=3
pad_types = ['zero', 'replicate']
windows = ['lung', 'mediastinal', 'shifted_lung']
project_name = 'covid19_seg'
width_irc = (12,192,192)

In [5]:
model_path = 'saved-models/2020-11-22_11.01.15.best.state'

In [ ]:
run_name = 'unet3d train run p2'

In [1]:
run('training.CovidSegmentationTrainingApp', f'--epochs={epochs}', f'--batch-size={batch_size}', 
    f'--steps-per-epoch={steps_per_epoch}', f'--val-cadence={val_cadence}', f'--recall-priority={recall_priority}',
    f'--depth={unet_depth}', f'--pad-type={pad_types[1]}', '--augmented', f'--model-path={model_path}',
    '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}', 
    f'--project-name={project_name}', f'--run-name={run_name}')

# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

## Covid Seg Net

In [4]:
batch_size = 2
epochs = 600
steps_per_epoch=320
val_cadence = 5
depth = 3
windows = ['lung', 'mediastinal', 'shifted_lung']
project_name = 'covid19_seg'
width_irc=(16,128,128)

In [5]:
run_name = 'Train (320spe,2gpu,bs2x3,aug)'
notes = 'Train Run'

In [6]:
run('training_alpha.CovidSegmentationTrainingApp', f'--epochs={epochs}', f'--batch-size={batch_size}', f'--num-workers={8}',
    f'--ct-window={windows[2]}', f'--val-cadence={val_cadence}', f'--depth={depth}', '--augmented',
    '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}', f'--notes={notes}',
    f'--steps-per-epoch={steps_per_epoch}', f'--project-name={project_name}', f'--run-name={run_name}')


# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

2020-12-07 05:46:01,291 INFO     pid:26365 nb:005:run Running: training_alpha.CovidSegmentationTrainingApp(['--epochs=600', '--batch-size=2', '--num-workers=8', '--ct-window=shifted_lung', '--val-cadence=5', '--depth=3', '--augmented', '--width-irc', '16', '128', '128', '--notes=Train Run', '--steps-per-epoch=320', '--project-name=covid19_seg', '--run-name=Train (320spe,2gpu,bs2x3,aug)']).main()
2020-12-07 05:46:03,145 INFO     pid:26365 wandb:606:_apply_source_login setting login settings: {}
wandb: Currently logged in as: peterbacalso (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


2020-12-07 05:46:05,664 INFO     pid:26365 training_alpha:217:init_model Using CUDA; 2 devices.
2020-12-07 05:46:09,100 INFO     pid:26365 modules.dsets:374:__init__ TrainingV2Covid2dSegmentationDataset: training mode, 159 uid's, 320 steps_per_epoch
2020-12-07 05:46:09,104 INFO     pid:26365 modules.dsets:447:__init__ TrainingV2Covid2dSegmentationDataset: (16, 128, 128) width_irc
2020-12-07 05:46:09,107 INFO     pid:26365 modules.dsets:374:__init__ Covid2dSegmentationDataset: validation mode, 40 uid's, 40 steps_per_epoch
2020-12-07 05:46:09,109 INFO     pid:26365 training_alpha:516:main Starting CovidSegmentationTrainingApp, Namespace(augment_flip=False, augment_noise=False, augment_offset=False, augment_rotate=False, augment_scale=False, augmented=True, batch_size=2, ct_window='shifted_lung', depth=3, epochs=600, model_path=None, notes='Train Run', num_workers=8, project_name='covid19_seg', run_name='Train (320spe,2gpu,bs2x3,aug)', steps_per_epoch=320, val_cadence=5, width_irc=['16', 

epoch,loss/trn,dice_loss/trn,ce_loss/trn,loss/val,dice_loss/val,ce_loss/val,metrics_val/miss_rate,metrics_val/fp_to_mask_ratio,pr_val/precision,pr_val/recall,pr_val/f1_score
1,0.8241,0.9000,0.6469,0.8759,0.9837,0.6245,0.3368,77.7321,0.0085,0.6632,0.0167
5,0.7393,0.8703,0.4335,0.8070,0.9706,0.4253,0.5996,23.9239,0.0165,0.4004,0.0316
10,0.6459,0.7584,0.3834,0.6326,0.7579,0.3404,0.4953,1.3585,0.2709,0.5047,0.3525
15,0.5639,0.6473,0.3693,0.6078,0.7267,0.3303,0.5403,0.7024,0.3955,0.4597,0.4252
20,0.5506,0.6306,0.3639,0.6071,0.7277,0.3257,0.6030,0.4227,0.4843,0.3970,0.4363
25,0.5145,0.5806,0.3604,0.5771,0.6856,0.3238,0.5651,0.4504,0.4912,0.4349,0.4613
30,0.5028,0.5667,0.3538,0.5637,0.6673,0.3220,0.5828,0.2854,0.5938,0.4172,0.4901
35,0.5105,0.5777,0.3537,0.5457,0.6418,0.3216,0.5176,0.3999,0.5467,0.4824,0.5126
40,0.5043,0.5695,0.3521,0.5184,0.6027,0.3217,0.4506,0.5953,0.4800,0.5494,0.5124
45,0.5027,0.5675,0.3513,0.5131,0.5949,0.3222,0.3675,0.8559,0.4249,0.6325,0.5083


2020-12-07 05:47:37,310 INFO     pid:26365 training_alpha:384:log_metrics E1 CovidSegmentationTrainingApp
2020-12-07 05:47:37,315 INFO     pid:26365 training_alpha:415:log_metrics E1 trn      0.8241 loss, 0.9000 dice loss, 0.6469 ce loss, 0.0632 precision, 0.6339 recall, 0.1150 f1 score 0.3661 miss rate 9.3905 fp to label ratio
2020-12-07 05:49:01,546 INFO     pid:26365 training_alpha:384:log_metrics E1 CovidSegmentationTrainingApp
2020-12-07 05:49:01,552 INFO     pid:26365 training_alpha:426:log_metrics E1 val      0.8759 loss, 0.9837 dice loss, 0.6245 ce_loss, 0.0085 precision, 0.6632 recall, 0.0167 f1 score 0.3368 miss rate 77.7321 fp to label ratio
2020-12-07 05:49:04,149 INFO     pid:26365 training_alpha:502:save_model Saved model params to saved-models/2020-12-07_05.46.02.320.state
2020-12-07 05:49:04,158 INFO     pid:26365 training_alpha:507:save_model Saved model params to saved-models/2020-12-07_05.46.02.best.state
2020-12-07 05:49:04,172 INFO     pid:26365 training_alpha:512:

loss/trn,0.27196
dice_loss/trn,0.24817
ce_loss/trn,0.32747
metrics_trn/miss_rate,0.1567
metrics_trn/fp_to_mask_ratio,0.08876
pr_trn/precision,0.90477
pr_trn/recall,0.8433
pr_trn/f1_score,0.87296
loss/val,0.32352
dice_loss/val,0.32589
ce_loss/val,0.31798


loss/trn,█▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁
dice_loss/trn,█▆▅▅▅▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁
ce_loss/trn,█▅▄▃▃▃▃▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁
metrics_trn/miss_rate,███▇▆▆▆▆▅▄▄▄▄▄▄▄▄▄▃▄▃▃▃▃▃▃▃▃▃▂▂▂▂▁▂▁▁▁▁▁
metrics_trn/fp_to_mask_ratio,█▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pr_trn/precision,▁▄▅▆▆▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█████████
pr_trn/recall,▁▁▁▂▃▃▃▃▄▅▅▅▅▅▅▅▅▅▆▅▆▆▆▆▆▆▆▆▆▇▇▇▇█▇█████
pr_trn/f1_score,▁▂▃▄▄▅▅▅▅▆▆▅▆▆▆▆▆▆▆▆▆▇▆▆▆▆▇▇▇▇▇▇▇███████
loss/val,█▅▄▃▃▃▆▃▃▄▃▄▄▄▅▂▂▃▂▃▂▃▃▂▂▂▃▂▂▂▂▂▂▁▁▁▁▁▁▁
dice_loss/val,█▅▅▄▄▃▇▄▃▄▃▅▄▄▆▃▂▄▂▃▃▃▃▂▂▂▃▂▃▂▂▂▂▁▁▂▁▁▁▁
ce_loss/val,█▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: wandb version 0.10.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


RuntimeError: No active exception to reraise